In [2]:
import copy
import cv2 as cv
import imageio
from multiprocessing import Pool
import numpy as np
import os
import open3d as o3d
import pickle
import scipy
import shutil
from tqdm import tqdm

In [3]:
# Scaling coefficient for depth, the same as in TUM
DEPTH_SCALE_FACTOR = 5000
SETUP_CONFIG = 'bandeja_standard.pickle'

# Folder where the data in TUM format will be put
final_folder = '2021-03-01-15-37-25'

azure_depth_folder = '../bag-extractor/2021-03-01-15-37-25/_azure_depth_image_raw/'
smartphone_folder = '../bag-extractor/2021-03-01-15-37-25/smartphone_video_frames/'

In [4]:
depth_ts = np.array([int(file.split('.')[0]) for file in os.listdir(azure_depth_folder) 
                     if file.endswith('.npy')])
depth_ts.sort()

rgb_ts = np.array([int(file.split('.')[0]) for file in sorted(os.listdir(smartphone_folder)) 
                   if file.endswith('.png')])
rgb_ts.sort()

print('Depth timestamps from {1} to {2} (cnt {0})'.format(len(depth_ts), depth_ts[0], depth_ts[-1]))
print('RGB timestamps from {1} to {2} (cnt {0})'.format(len(rgb_ts), rgb_ts[0], rgb_ts[-1]))

# Build correspondences between depth and rgb by nearest neighbour algorithm
rgbd_pairs = []
for depth_t in depth_ts:
    closest_rgb_t = min(rgb_ts, key=lambda x: abs(depth_t - x))
    rgbd_pairs.append((closest_rgb_t, depth_t))
    
# Prepare folder infrastructure
if os.path.exists(final_folder):
    shutil.rmtree(final_folder)
os.mkdir(final_folder)
os.mkdir(os.path.join(final_folder, 'depth'))
os.mkdir(os.path.join(final_folder, 'rgb'))

FileNotFoundError: [Errno 2] No such file or directory: '../bag-extractor/2021-03-01-15-37-25/_azure_depth_image_raw/'

In [5]:
# Point cloud from depth (by Konstantin)
def pointcloudify_depth(depth, intrinsics, dist_coeff, undistort=True):
    shape = depth.shape[::-1]
    
    if undistort:
        undist_intrinsics, _ = cv.getOptimalNewCameraMatrix(intrinsics, dist_coeff, shape, 1, shape)
        inv_undist_intrinsics = np.linalg.inv(undist_intrinsics)

    else:
        inv_undist_intrinsics = np.linalg.inv(intrinsics)

    if undistort:
        # undist_depthi = cv.undistort(depthi, intrinsics, dist_coeff, None, undist_intrinsics)
        map_x, map_y = cv.initUndistortRectifyMap(intrinsics, dist_coeff, None
                                                  , undist_intrinsics, shape, cv.CV_32FC1)
        undist_depth = cv.remap(depth, map_x, map_y, cv.INTER_NEAREST)

    # Generate x,y grid for H x W image
    grid_x, grid_y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    grid = np.concatenate([np.expand_dims(grid_x, -1),
                           np.expand_dims(grid_y, -1)], axis=-1)

    grid = np.concatenate([grid, np.ones((shape[1], shape[0], 1))], axis=-1)

    # To normalized image coordinates
    local_grid = inv_undist_intrinsics @ grid.reshape(-1, 3).transpose()  # 3 x H * W

    # Raise by undistorted depth value from image plane to local camera space
    if undistort:
        local_grid = local_grid.transpose() * np.expand_dims(undist_depth.reshape(-1), axis=-1)

    else:
        local_grid = local_grid.transpose() * np.expand_dims(depth.reshape(-1), axis=-1)
        
    return local_grid.astype(np.float32)


def project_pcd_to_depth(pcd, undist_intrinsics, img_size): 
    I = np.zeros(img_size, np.float32)
    h, w = img_size
    for P in pcd.points:
        d = np.linalg.norm(P)
        P = P / P[2]
        p = undist_intrinsics @ P
        x, y = int(np.round(p[0])), int(np.round(p[1]))
        if x >= 0 and x < w and y >= 0 and y < h:
            I[y, x] = d
            
    return I


def smooth_depth(depth):
    MAX_DEPTH_VAL = 1e5
    KERNEL_SIZE = 11
    depth[depth == 0] = MAX_DEPTH_VAL
    smoothed_depth = scipy.ndimage.minimum_filter(depth, KERNEL_SIZE)
    smoothed_depth[smoothed_depth == MAX_DEPTH_VAL] = 0
    return smoothed_depth


def align_rgb_depth(rgb, depth, roi, config_file=SETUP_CONFIG):
    with open(config_file, 'rb') as config:
        config_dict = pickle.load(config)

    # Undistort rgb image
    undist_rgb = cv.undistort(rgb, config_dict['rgb']['dist_mtx'], config_dict['rgb']['dist_coef'],
                              None, config_dict['rgb']['undist_mtx'])
    
    # Create point cloud from depth
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointcloudify_depth(depth, config_dict['depth']['dist_mtx'],
                                                                config_dict['depth']['dist_coef']))

    # Align point cloud with depth reference frame
    pcd.transform(config_dict['T'])
    
    # Project aligned point cloud to rgb
    aligned_depth = project_pcd_to_depth(pcd, config_dict['rgb']['undist_mtx'], rgb.shape[:2])
    
    smoothed_aligned_depth = smooth_depth(aligned_depth)
    x, y, w, h = roi

    depth_res = smoothed_aligned_depth[y:y+h, x:x+w]
    rgb_res = undist_rgb[y:y+h, x:x+w]
    return rgb_res, depth_res


In [37]:
def process_pair(rgbd_pair):
    rgb_image = cv.imread(os.path.join(smartphone_folder, str(rgbd_pair[0]) + '.png'))
    depth_array = np.load(os.path.join(azure_depth_folder, str(rgbd_pair[1]) + '.npy'))
    
    rgb_image_aligned, depth_array_aligned = align_rgb_depth(rgb_image, depth_array, (0, 0, 1920, 1080))
    
    # Save rgb as 8-bit png
    cv.imwrite(os.path.join(final_folder, 'rgb', str(rgbd_pair[0] / 1e9) + '.png'), rgb_image_aligned)
    
    # Save depth as 16-bit unsigned int with scale factor
    depth_array_aligned = (depth_array_aligned * DEPTH_SCALE_FACTOR).astype(np.uint16)
    imageio.imwrite(os.path.join(final_folder, 'depth', str(rgbd_pair[1] / 1e9) + '.png'), depth_array_aligned)

# Copy pairs of rgb and depth to final dir
with Pool() as pool: 
    pool.map(process_pair, rgbd_pairs)

# Produce file with associations between rgb and depth files
with open(os.path.join(final_folder, 'association.txt'), 'w') as association_file:
    for rgbd_pair in rgbd_pairs:
        association_file.write('{0} {1} {2} {3}\n'
                               .format(rgbd_pair[0] / 1e9, os.path.join('rgb', str(rgbd_pair[0] / 1e9) + '.png'),
                                       rgbd_pair[1] / 1e9, os.path.join('depth', str(rgbd_pair[1] / 1e9) + '.png')))